# Spectractor Launcher

Goal is to process a series of images in order to produce spectra

- author Sylvie Dagoret-Campagne
- date : March 29th 2018
- update : April 9th 2018 


In [191]:
theNotebook="Run_Spectractor"

# Spectractor Launcher

In [192]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Required packages 

In [193]:
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd

In [194]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [195]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),   #defines the default image size
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [196]:
plt.rcParams.update({'figure.max_open_warning': 0}) # to see a large number of files

## Tools

In [197]:
#sys.path.append("../common_tools/")
#from common_notebook import *
#init_notebook()

## Spectractor 

In [198]:
PATH_SPECTRACTOR='../../Spectractor'

In [199]:
sys.path.append(PATH_SPECTRACTOR)

In [200]:
from spectractor import *

In [201]:
from tools import *
from holo_specs import *
from targets import *
from optics import *
import parameters 

# Configuration

## Input Directory

### HD111980 is missing in logbook

In [202]:
if 0:
    inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
    MIN_IMGNUMBER=58
    #MAX_IMGNUMBER=80  # temporary
    MAX_IMGNUMBER=190
    date="20170530"
    object_name='HD111980'
    tag = "reduc"
    SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
    SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
    NBIMGPERROW=2
    subdir=os.path.basename(inputdir)

### So use HD205905

In [203]:
if 0:
    #inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
    inputdir="/Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_30may17"
    #MIN_IMGNUMBER=208
    MIN_IMGNUMBER=230
    MAX_IMGNUMBER=284
    date="20170530"
    object_name='HD205905'
    tag = "reduc"
    SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
    SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
    NBIMGPERROW=2
    subdir=os.path.basename(inputdir)

In [204]:
# Reconstruction April 9th 2018 by Sylvie
#inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
inputdir="/Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_28may17"
#MIN_IMGNUMBER=208
MIN_IMGNUMBER=56
MAX_IMGNUMBER=135
date="20170528"
object_name='HD185975'
tag = "reduc"
SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
NBIMGPERROW=2
subdir=os.path.basename(inputdir)
skipped_indexes= [69,73,123,130,131,132]

# Output directory

In [205]:
dir_top_images="images_notebooks/"+theNotebook ### Top directory to save images
ensure_dir(dir_top_images) # create that directory
outputdir="./spectra"
outputdir=os.path.join(outputdir,subdir)
#ensure_dir(outputdir)

In [206]:
#infile_fullpath=os.path.join(PATH_SPECTRACTOR,infile)

In [207]:
sys.path.append(PATH_SPECTRACTOR)

In [208]:
from spectractor import *

# Logbook

In [209]:
#file_logbook_csv='../common_info/ctiofulllogbook_jun2017_v3.csv'
#file_logbook_xlsx='../common_info/ctiofulllogbook_jun2017_v3.xlsx'
file_logbook_xlsx='../common_info/ctiofulllogbook_jun2017_v4.xlsx'

In [210]:
#df=pd.read_csv(file_logbook_csv,index_col=0)
df=pd.read_excel(file_logbook_xlsx,index_col=0)

In [211]:
df.head()

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip
31,2017-06-02T04:26:04.602,data_01jun17,15.0,HD185975,dia,HoloPhAg,1.919,120.0,12400.0,0.759,...,12.8,22.0,21.3,reduc_20170601_015.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
30,2017-06-02T04:29:01.302,data_01jun17,16.0,HD185975,dia,HoloPhAg,1.918,120.0,12400.0,0.759,...,12.9,22.0,12.1,reduc_20170601_016.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
99,2017-06-02T04:31:58.001,data_01jun17,17.0,HD185975,dia,HoloPhAg,1.917,120.0,12400.0,0.745,...,13.0,22.0,14.3,reduc_20170601_017.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
50,2017-06-02T04:34:52.548,data_01jun17,18.0,HD185975,dia,HoloPhAg,1.917,120.0,12400.0,0.742,...,13.2,21.0,20.2,reduc_20170601_018.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
6,2017-06-02T04:37:47.745,data_01jun17,19.0,HD185975,dia,HoloPhAg,1.916,120.0,12400.0,0.728,...,13.2,21.0,24.4,reduc_20170601_019.fits,NaN,NaN,NaN,NaN,mesure holo,NaN


# Extract the relevant data and sort by the column index

In [212]:
all_obs=df.loc[(df["subdir"]==subdir) & (df["object"]==object_name),:].sort_values(by=['index'])

In [213]:
all_obs.head()

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip
78,2017-05-29T04:16:11.205,data_28may17,56.0,HD185975,dia,HoloAmAg,1.931,30.0,12550.0,1.45,...,6.8,38.0,35.6,reduc_20170528_056.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
63,2017-05-29T04:18:29.109,data_28may17,57.0,HD185975,dia,Thor300,1.929,30.0,12550.0,1.45,...,6.8,38.0,33.6,reduc_20170528_057.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
7,2017-05-29T04:19:51.453,data_28may17,58.0,HD185975,dia,HoloPhP,1.929,30.0,12550.0,1.45,...,6.9,38.0,28.1,reduc_20170528_058.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
73,2017-05-29T04:21:13.848,data_28may17,59.0,HD185975,dia,HoloPhAg,1.928,30.0,12550.0,1.45,...,6.6,39.0,24.8,reduc_20170528_059.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
98,2017-05-29T04:27:42.796,data_28may17,60.0,HD185975,dia,Thor300,1.924,30.0,12550.0,1.45,...,6.2,41.0,29.2,reduc_20170528_060.fits,780.0,650.0,50.0,100.0,atmosphère,NaN


# Extract relevant info for Spectractor

In [214]:
filex0y0dxdy=all_obs[['index','file','Obj-posXpix','Obj-posYpix','Dx','Dy']]

In [215]:
filex0y0dxdy.head()

,index,file,Obj-posXpix,Obj-posYpix,Dx,Dy
78,56.0,reduc_20170528_056.fits,780.0,650.0,50.0,100.0
63,57.0,reduc_20170528_057.fits,780.0,650.0,50.0,100.0
7,58.0,reduc_20170528_058.fits,780.0,650.0,50.0,100.0
73,59.0,reduc_20170528_059.fits,780.0,650.0,50.0,100.0
98,60.0,reduc_20170528_060.fits,780.0,650.0,50.0,100.0


In [216]:
filex0y0dxdy.tail()

,index,file,Obj-posXpix,Obj-posYpix,Dx,Dy
146,166.0,reduc_20170528_166.fits,NaN,NaN,NaN,NaN
155,167.0,reduc_20170528_167.fits,NaN,NaN,NaN,NaN
143,168.0,reduc_20170528_168.fits,NaN,NaN,NaN,NaN
107,169.0,reduc_20170528_169.fits,NaN,NaN,NaN,NaN
85,170.0,reduc_20170528_170.fits,NaN,NaN,NaN,NaN


### Select a few spectra for debugging


- def fit_multigauss_and_bgd(x,y,guess=[10,1000,1,0,0,1],bounds=(-np.inf,np.inf),sigma=None):
- maxfev=100000 au lieu de  maxfev=10000



In [217]:
filex0y0dxdy=filex0y0dxdy.loc[(filex0y0dxdy["index"]>=MIN_IMGNUMBER) &  (filex0y0dxdy["index"]<=MAX_IMGNUMBER),:]

In [218]:
filex0y0dxdy

,index,file,Obj-posXpix,Obj-posYpix,Dx,Dy
78,56.0,reduc_20170528_056.fits,780.0,650.0,50.0,100.0
63,57.0,reduc_20170528_057.fits,780.0,650.0,50.0,100.0
7,58.0,reduc_20170528_058.fits,780.0,650.0,50.0,100.0
73,59.0,reduc_20170528_059.fits,780.0,650.0,50.0,100.0
98,60.0,reduc_20170528_060.fits,780.0,650.0,50.0,100.0
56,61.0,reduc_20170528_061.fits,780.0,650.0,50.0,100.0
136,62.0,reduc_20170528_062.fits,780.0,650.0,50.0,100.0
36,63.0,reduc_20170528_063.fits,780.0,650.0,50.0,100.0
24,64.0,reduc_20170528_064.fits,780.0,650.0,50.0,100.0
135,65.0,reduc_20170528_065.fits,780.0,650.0,50.0,100.0


### Setting the parameters of Spectractor

In [ ]:
parameters.DEBUG = False
parameters.VERBOSE = True

In [ ]:
for idx, row in filex0y0dxdy.iterrows():
    full_filename=os.path.join(inputdir,row['file'])
    num,x0,y0=row['index'],row['Obj-posXpix'], row['Obj-posYpix']
    print '**********************************************************************************'
    print '*********** spectrum id=',index, ' num =',num, ' x0=',x0,' y0=',y0,' *************'
    print '**********************************************************************************'
    # test if file already exists Spectractor skip it
    if num in skipped_indexes:
        print '>>>>>>>>>>>>> SKIP spectrum num = ', num
    else:
        Spectractor(full_filename,outputdir,[int(x0),int(y0)],object_name)

2018-04-09 11:54:10  spectractor Spectractor          INFO   
	Start SPECTRACTOR
2018-04-09 11:54:10  Image      load                 INFO   
	Loading image /Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_28may17/reduc_20170528_056.fits...
2018-04-09 11:54:10  Image      load                 INFO   
	Image loaded
2018-04-09 11:54:10  Image      load                 INFO   
	Loading disperser HoloAmAg...


**********************************************************************************
*********** spectrum id= 105  num = 56.0  x0= 780.0  y0= 650.0  *************
**********************************************************************************
Load disperser HoloAmAg:
	from ../../Spectractor/dispersers/HoloAmAg
Hologram characteristics:
	N = 348.73 +/- 1.14 grooves/mm at plate center
	Plate center at x0 = 774.0 and y0 = 690.7 with average tilt of -1.9 degrees
	Hologram center at x0 = 1070.9 and y0 = 586.1 with average tilt of -1.9 degrees
Order  0 position at x0 = 805.1 and y0 = 595.0
Order +1 position at x0 = 1336.6 and y0 = 577.3
Distance between the orders: 531.79 pixels (12.76 mm)


2018-04-09 11:54:14  spectractor Spectractor          INFO    !!!!!! Spectrum file file ./spectra/data_28may17/reduc_20170528_056_spectrum.fits of size 28800 already exists, thus SKIP the reconstruction ...
2018-04-09 11:54:14  spectractor Spectractor          INFO   
	Start SPECTRACTOR
2018-04-09 11:54:14  Image      load                 INFO   
	Loading image /Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_28may17/reduc_20170528_057.fits...
2018-04-09 11:54:14  Image      load                 INFO   
	Image loaded
2018-04-09 11:54:14  Image      load                 INFO   
	Loading disperser Thor300...


 MAIN_ID        RA           DEC      ... SP_QUAL_373    SP_BIBCODE_373  
             "h:m:s"       "d:m:s"    ...                                
--------- ------------- ------------- ... ----------- -------------------
HD 185975 20 28 18.7284 -87 28 19.941 ...           C 1975MSS...C01....0H
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_001.fits
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_002.fits
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_003.fits
**********************************************************************************
*********** spectrum id= 105  num = 57.0  x0= 780.0  y0= 650.0  *************
**********************************************************************************
Load disperser Thor300:
	from ../../Spectractor/dispersers/Thor300
Grating characteristics:
	N = 299.51 +/- 0.99 grooves/mm
	Average tilt of -0.6 degrees
